In [1]:
import torch
import numpy as np
import torch.nn as nn
import pandas as pd
import re
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import r2_score
import torchvision.transforms as transforms
from torchvision.io import read_image
from torch.utils.data import Dataset
from sklearn.model_selection import KFold
import random
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import os
from torch.utils.data import DataLoader

In [ ]:
gen_path = 'full_data/'
case_name = '2D_CNN_100_0_001_baseline'

cnn_1d_path = gen_path + case_name+'/' #path to the folder where you store the data

in_num = 3 #number of runs for each cross-validation fold
num_param = 4 #number of output parameters

k_folds = [[42,12],[612,45],[72,172],[871,48],[52,134]] #cross-validation folds you used during training

all_folds = []

for fold in k_folds:
    
    split_path = cnn_1d_path+'split_'+ str(fold[0])+'_'+str(fold[1])+ '/'
    
    mae_trn = np.zeros((1,num_param))
    mae_vld = np.zeros((1,num_param))
    mae_tst = np.zeros((1,num_param))

    rmse_trn = np.zeros((1,num_param))
    rmse_vld = np.zeros((1,num_param))
    rmse_tst = np.zeros((1,num_param))

    r2_trn = np.zeros((1,num_param))
    r2_vld = np.zeros((1,num_param))
    r2_tst = np.zeros((1,num_param))
    
    for init_name in range(0,in_num):
        
        init_path = split_path + str(init_name) +'\\'

        a = ['Cu','Ni','Cr','NO3']

        y_trn_true = np.array(pd.read_csv(init_path + 'Y_true_trn.csv').iloc[:,1:])
        y_trn_predict = np.array(pd.read_csv(init_path + 'Y_out_trn.csv').iloc[:,1:])
        
        y_vld_true = np.array(pd.read_csv(init_path + 'Y_true_vld.csv').iloc[:,1:])
        y_vld_predict = np.array(pd.read_csv(init_path + 'Y_out_vld.csv').iloc[:,1:])
        
        y_tst_true = np.array(pd.read_csv(init_path + 'Y_true_tst.csv').iloc[:,1:])
        y_tst_predict = np.array(pd.read_csv(init_path + 'Y_out_tst.csv').iloc[:,1:])
        
        mae_trn += mean_absolute_error(y_trn_true, y_trn_predict, multioutput='raw_values')
        mae_vld += mean_absolute_error(y_vld_true, y_vld_predict, multioutput='raw_values')
        mae_tst += mean_absolute_error(y_tst_true, y_tst_predict, multioutput='raw_values')

        rmse_trn += np.sqrt(mean_squared_error(y_trn_true, y_trn_predict, multioutput='raw_values'))
        rmse_vld += np.sqrt(mean_squared_error(y_vld_true, y_vld_predict, multioutput='raw_values'))
        rmse_tst += np.sqrt(mean_squared_error(y_tst_true, y_tst_predict, multioutput='raw_values'))

        r2_trn += r2_score(y_trn_true, y_trn_predict, multioutput='raw_values')
        r2_vld += r2_score(y_vld_true, y_vld_predict, multioutput='raw_values')
        r2_tst += r2_score(y_tst_true, y_tst_predict, multioutput='raw_values')
    
    mae_trn /= in_num
    mae_vld /= in_num
    mae_tst /= in_num
    
    mae_trn = pd.DataFrame(mae_trn, columns=a)
    mae_vld = pd.DataFrame(mae_vld, columns=a)
    mae_tst = pd.DataFrame(mae_tst, columns=a)

    rmse_trn /= in_num
    rmse_vld /= in_num
    rmse_tst /= in_num

    rmse_trn = pd.DataFrame(rmse_trn, columns=a)
    rmse_vld = pd.DataFrame(rmse_vld, columns=a)
    rmse_tst = pd.DataFrame(rmse_tst, columns=a)

    r2_trn /= in_num
    r2_vld /= in_num
    r2_tst /= in_num
    
    r2_trn = pd.DataFrame(r2_trn, columns=a)
    r2_vld = pd.DataFrame(r2_vld, columns=a)
    r2_tst = pd.DataFrame(r2_tst, columns=a)

    tst = [ mae_tst,rmse_tst, r2_tst]
    vld = [ mae_vld,rmse_vld, r2_vld]
    trn = [ mae_trn,rmse_trn, r2_trn]
    
    met_tst = pd.DataFrame({'tst_metric':['mae', 'rmse', 'r2']})
    met_vld = pd.DataFrame({'vld_metric':['mae', 'rmse', 'r2']})
    met_trn = pd.DataFrame({'trn_metric':['mae', 'rmse', 'r2']})
    
    split_met = pd.concat([met_tst,pd.concat(tst).reset_index(drop=True),met_vld,pd.concat(vld).reset_index(drop=True),met_trn,pd.concat(trn).reset_index(drop=True)], axis = 1)

    a = pd.DataFrame(np.array(['split_'+ str(fold[0])+'_'+str(fold[1]),' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ']).reshape(1,-1), columns = split_met.columns).reset_index(drop=True)
    
    all_folds.append(a.append(split_met))

In [ ]:
(pd.concat(all_folds)).to_csv(cnn_1d_path+case_name+'_Stat.csv')